# Import

In [91]:
import os, os.path
import pandas as pd
import numpy as np, os 
from tqdm import tqdm
from collections import Counter
import time
import sys

from torch.utils.data import Dataset, DataLoader
import torch


# Access to file

In [2]:
list_txt = os.listdir("train") # dir is your directory path
number_files = len(list_txt)
number_files

15594

In [3]:
type(list_txt)

list

In [4]:
index_txt = list_txt.index('0A0AA9C21C5D.txt')
index_txt                   

6804

In [5]:
os.getcwd()

'/home/arno/Documents/1_Study/Project/NLP_sentiment_pred'

In [6]:
def open_text(text):
    with open('train/'+text+'.txt') as f:
        text = f.read()
    # print out the first 100 characters
    return text

# Data prep

In [7]:
train_csv = pd.read_csv('train.csv')

In [8]:
print(train_csv.shape )
train_csv.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [9]:
train_csv.iloc[0]

id                                                         423A1CA112E2
discourse_id                                            1622627660524.0
discourse_start                                                     8.0
discourse_end                                                     229.0
discourse_text        Modern humans today are always on their phone....
discourse_type                                                     Lead
discourse_type_num                                               Lead 1
predictionstring      1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
Name: 0, dtype: object

In [10]:
test_names, test_texts = [], []
for f in os.listdir('test'):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,D46BCB48440A,"When people ask for advice,they sometimes talk..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D72CB1C11673,Making choices in life can be very difficult. ...
3,DF920E0A7337,Have you ever asked more than one person for h...
4,0FB0700DAF44,"During a group project, have you ever asked a ..."


# Train CSV review and creation of counters


In [ ]:
def creation_review():
    

In [11]:
def creation_count(train_csv):
    # list of counters
    lead_counts = Counter()
    position_counts = Counter()
    claim_counts = Counter()
    counterclaim_counts = Counter()
    rebuttal_counts = Counter()
    evidence_counts = Counter()
    conclustate_counts = Counter()

    for i in range(len(train_csv.discourse_id)):
        text = train_csv.discourse_text[i]
        text = text.split(' ')
        if train_csv.discourse_type[i] == 'Lead':
            for words_ind in range(len(text)):
                lead_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Position':
            for words_ind in range(len(text)):
                position_counts[text[words_ind-1]] += 1
            
        elif train_csv.discourse_type[i] == 'Claim':
            for words_ind in range(len(text)):
                claim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Counterclaim':
            for words_ind in range(len(text)):
                counterclaim_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Rebuttal':
            for words_ind in range(len(text)):
                rebuttal_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Evidence':
            for words_ind in range(len(text)):
                evidence_counts[text[words_ind-1]] += 1

        elif train_csv.discourse_type[i] == 'Concluding Statement':
            for words_ind in range(len(text)):
                conclustate_counts[text[words_ind-1]] += 1
    
    return lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts


In [12]:
lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts = creation_count(train_csv)

In [30]:
total_count = lead_counts+position_counts+claim_counts+counterclaim_counts+rebuttal_counts+evidence_counts+conclustate_counts 

In [31]:
def counter_ratio (train_csv):
    lead_counts, position_counts, claim_counts, counterclaim_counts, rebuttal_counts, evidence_counts, conclustate_counts = creation_count(train_csv)
    total_count = lead_counts+position_counts+claim_counts+counterclaim_counts+rebuttal_counts+evidence_counts+conclustate_counts 
    # lead count
    lead_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            lead_ratio = lead_counts[term] / float(total_count[term]+1)
            lead_counts_ratios[term] = lead_ratio
            
    # position count
    position_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            position_ratio = position_counts[term] / float(total_count[term]+1)
            position_counts_ratios[term] = position_ratio
            
    #claim count
    claim_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            claim_ratio = claim_counts[term] / float(total_count[term]+1)
            claim_counts_ratios[term] = claim_ratio
            
    #counter claim count
    counterclaim_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            counterclaim_ratio = counterclaim_counts[term] / float(total_count[term]+1)
            counterclaim_counts_ratios[term] = counterclaim_ratio
            
    #rebuttal count
    rebuttal_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            rebuttal_ratio = rebuttal_counts[term] / float(total_count[term]+1)
            rebuttal_counts_ratios[term] = rebuttal_ratio
            
    #evidence count
    evidence_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            evidence_ratio = evidence_counts[term] / float(total_count[term]+1)
            evidence_counts_ratios[term] = evidence_ratio
            
    #concluding statement count
    conclustate_counts_ratios = Counter()
    for term,cnt in list(total_count.most_common()):
        if(cnt > 50):
            conclustate_ratio = conclustate_counts[term] / float(total_count[term]+1)
            conclustate_counts_ratios[term] = conclustate_ratio
    
    return lead_counts_ratios, position_counts_ratios, claim_counts_ratios, counterclaim_counts_ratios,rebuttal_counts_ratios, evidence_counts_ratios, conclustate_counts_ratios

In [32]:
lead_counts_ratios, position_counts_ratios, claim_counts_ratios, counterclaim_counts_ratios,rebuttal_counts_ratios, evidence_counts_ratios, conclustate_counts_ratios = counter_ratio(train_csv)

# Convert to log

test on lead_count

In [33]:
def log_transform(counter_ratios):
    for word,ratio in counter_ratios.most_common():
        if ratio > 0:
            counter_ratios[word] = np.log(ratio)
    return counter_ratios

In [34]:
lead_counts_log_ratios = log_transform(lead_counts_ratios)

In [35]:
list(reversed(lead_counts_log_ratios.most_common()))

[('conclusion,', -7.193685818395112),
 ('Overall,', -5.857933154483459),
 ('run-off', -5.58724865840025),
 ('hovering', -5.5093883366279774),
 ('sensor', -5.3981627015177525),
 ('Heidrun', -5.272999558563747),
 ('defy', -5.214935757608986),
 ('winner', -5.191103282240888),
 ('lava', -5.1647859739235145),
 ('Second,', -5.135798437050262),
 ('butte', -5.1328529268205045),
 ('Acropolis', -5.075173815233827),
 ('dome', -5.0689042022202315),
 ('Carlos', -5.056245805348308),
 ('meters', -5.0106352940962555),
 ('Another', -5.0056227522727115),
 ('castle', -5.003946305945459),
 ('Next,', -4.997212273764115),
 ('interaction', -4.955827057601261),
 ('Arturo', -4.941642422609305),
 ('credit', -4.927253685157205),
 ('survivable', -4.927253685157205),
 ('Policy', -4.924835304292923),
 ('California,', -4.897839799950911),
 ('airplanes', -4.882801922586371),
 ('legislatures', -4.882801922586371),
 ('fiasco', -4.875197323201151),
 ('Garvin', -4.867534450455582),
 ('pixel', -4.863680881139593),
 ('orga

# transform words to number

In [52]:
vocab = set(total_count.keys())

In [53]:
vocab_size = len(vocab)
print(vocab_size)

125829


In [54]:
layer_0 = np.zeros((1,vocab_size))

In [55]:
layer_0.shape

(1, 125829)

In [56]:
def dict_word (vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word] = i
    return word2index

In [57]:
word2index = dict_word(vocab)

In [58]:
word2index

{'': 0,
 'inthepast': 1,
 "road'!!!": 2,
 'world.\n\nI': 3,
 'cable,': 4,
 'engrossed': 5,
 'Adventures': 6,
 'accomodations': 7,
 'immediately\xa0going': 8,
 'institution': 9,
 'particiate': 10,
 'road-tested': 11,
 "other's.\n\nThe": 12,
 'rapidly"': 13,
 'hookup': 14,
 'thiings': 15,
 'personially': 16,
 '"BFF"': 17,
 'gaming': 18,
 'becauce': 19,
 'dealers,': 20,
 'driver.\n\nKids': 21,
 'case,the': 22,
 'expessed.': 23,
 'access,and': 24,
 'landscapre.': 25,
 'emoions.': 26,
 'jive': 27,
 'immeasurable,': 28,
 'embarasment.': 29,
 "structure's": 30,
 'statewise': 31,
 'tranportation': 32,
 'challanged': 33,
 'Seahawks': 34,
 'Exorcist': 35,
 'life".': 36,
 'lolwer': 37,
 'single-sided': 38,
 'complying': 39,
 'testing.\n': 40,
 'severity': 41,
 'problom.': 42,
 'picked?': 43,
 'abke': 44,
 '154': 45,
 'way.\n\nIf': 46,
 'problematic.': 47,
 'needs.\n\nA': 48,
 'hostest': 49,
 'Low': 50,
 'salesmen': 51,
 'SAT.': 52,
 'home.\n\nIts': 53,
 'chief.': 54,
 'intonation': 55,
 'irrleven

In [63]:
def update_input_layer (review):
    global layer_0
    
    # set the layer to 0
    layer_0 *= 0
    
    for word in review.split(' '):
        layer_0[0][word2index[word]] +=1

In [64]:
train_csv.loc[0][4]

'Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.'

In [80]:
update_input_layer(train_csv.loc[0][4])
layer_0

array([[0., 0., 0., ..., 0., 0., 0.]])

In [66]:
def get_target_for_label(label):
    """Convert a label to `0` or `1`.
    Args:
        label(string) - Either "POSITIVE" or "NEGATIVE".
    Returns:
        `0` or `1`.
    """
    if(label == 'Lead'):
        return 1
    elif(label == 'Position'):
        return 2
    elif(label == 'Claim'):
        return 3
    elif(label == 'Counterclaim'):
        return 4
    elif(label == 'Rebuttal'):
        return 5
    elif(label == 'Evidence'):
        return 6
    elif(label == 'Concluding Statement'):
        return 7

In [67]:
train_csv.loc[0][5]

'Lead'

In [68]:
get_target_for_label(train_csv.loc[0][5])

1

# Build NN

In [ ]:
class Sentiment_NN:
    def __init__(self, reviews, labels, hidden_nodes=10, learning_rate=0.1):
        np.random.seed(1)
        
        self.pre_process_data(reviews, labels)
        
        self.init_network(len(slef.review_vocab), hidden_nodes, 1, learning_rate)
        
    def pre_process_data(self, reviews, labels):
        review_vocab = set()
        for review in train_csv['discourse_text']:
            for word in review.split(' '):
                review_vocab.add(word)
        self.review_vocab = list(review_vocab)
                
        label_vocab = set()
        for label in train_csv['discourse_type']:
            label_vocab.add(label)
        self.label_vocab = list(label_vocab)
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word]=i
        
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
            
    def init_network(self, input_node, hidden_node, output_node, learning_rate):
        
    def update_input_layer (review):

        # set the layer to 0
        self.layer_0 *= 0

        for word in review.split(' '):
            layer_0[0][self.word2index[word]] +=1
    
    def get_target_for_label(label):
    """Convert a label to `1`, `2`, '3', '4', '5', '6', '7'.
    Args:
        label(string) - Lead, Position, Claim, Counterclaim, Rebuttal, Evidence, Concluding Statement.
    Returns:
        `1`, `2`, '3', '4', '5', '6', '7'.
    """
        if(label == 'Lead'):
            return 1
        elif(label == 'Position'):
            return 2
        elif(label == 'Claim'):
            return 3
        elif(label == 'Counterclaim'):
            return 4
        elif(label == 'Rebuttal'):
            return 5
        elif(label == 'Evidence'):
            return 6
        elif(label == 'Concluding Statement'):
            return 7
    
    def sigmoid(self, x):
        return 1 / (1+np.exp(-1))
    
    def sigmoid2(sefl, output):
        return output * (1-output)
    
    def train (self, train_reviews, train_label):
        
    def test (self, test_reviews, test_label):
        
    def run (self)

        
        